In [1]:
import torch

In [18]:
from transformers import pipeline

generator = pipeline('text-generation', model="facebook/opt-iml-max-1.3b",
                    torch_dtype=torch.int32)

ValueError: Could not load model facebook/opt-iml-max-1.3b with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.opt.modeling_opt.OPTForCausalLM'>).

In [3]:
import json

with open('../msc/sample_data.jsonl', 'r') as f:
    dialogs = json.loads(f.read())

speakers = ['Speaker 1', 'Speaker 2']

In [4]:
from string import Template

prompts = [
    '$dialog\nSummarize the conversation between Speaker 1 and Speaker 2, including their facts:',
    '$dialog\nSummarize the conversation between Speaker 1 and Speaker 2:',
]

prompts = [Template(prompt) for prompt in prompts]

In [13]:
from tqdm import tqdm

results = []

for dialog in tqdm(dialogs):
    result = []
    for prompt in prompts:
        input_text = prompt.substitute({'dialog': dialog['dialog']})
        facts = generator(input_text, max_new_tokens=2048, top_k=3,
                        do_sample=True, temperature=0.9)[0]['generated_text']
        prompt_result = facts[len(input_text):]
        result.append(prompt_result) 
    results.append(result)

  0%|                                                    | 0/10 [00:00<?, ?it/s]


RuntimeError: MPS does not support min/max ops with int64 input

In [6]:
import numpy as np

In [8]:
torch.tensor(np.random.uniform(0, 100, size=(10, 10)), dtype=torch.int32, device=torch.device('mps')).min()

tensor(0, device='mps:0', dtype=torch.int32)